# Holder Wallet Conformance Flows (v3) - CTWalletSameDeferred

# 0.0 Initial setup

## 0.1 Setup conformance

In [27]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [28]:
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK
import uuid

# generate crypto seed
crypto_seed = b'helloworld'

key_did = KeyDid(seed=crypto_seed)

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9KbptQwoaj2VP1S6Ahzo7REFCT4NBTPYdQinCZbCcyoqWKi9Q2uEW36DNSXhCwiYnGz6BAZkzytQAEBE5cPidCGnadH4SsLDbSZeG2SEChrqvQpdK4Mk8H32vs3B5g8Wr7kcc


## 1.1 Initiate Credential Issuance

In [29]:
from ebsi_wallet.util import parse_query_string_parameters_from_url
from ebsi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type = CredentialTypes.CTWalletSameDeferred
qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)

[23:27:50] Credential offer URI:                                                                   ]8;id=134866;file:///tmp/ipykernel_59984/1703696794.py\1703696794.py]8;;\:]8;id=429268;file:///tmp/ipykernel_59984/1703696794.py#15\15]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/7bf85015-8992-4c5f-a6                 
           e1-02372ec3bd31                                                                                         

           Credential offer:                                                                       ]8;id=857568;file:///tmp/ipykernel_59984/1703696794.py\1703696794.py]8;;\:]8;id=432390;file:///tmp/ipykernel_59984/1703696794.py#18\18]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletSameDeferred'                                                                  
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'authorization_code': {                                                                         
                       'issuer_state':                                                                             
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNF                 
           ZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1                 
           BnWW95dHlrVVozZXlxaHQxajlLYnB0UXdvYWoyVlAxUzZBaHpvN1JFRkNUNE5CVFBZZFFpbkNaYkNjeW9xV0tpO                 
           VEydUVXMzZETlNYaEN3aVluR3o2QkFaa3p5dFFBRUJFNWNQaWRDR25hZEg0U3NMRGJTWmVHMlNFQ2hycXZRcGRL                 
           NE1rOEgzMnZzM0I1ZzhXcjdrY2MiLCJjcmVkZW50aWFsX3R5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                 
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRTYW1lRGVmZXJyZWQiXSwiaWF0IjoxNjg3MTk3NDcwLC                 
           JleHAiOjE2ODcxOTc3NzAsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY                 
           2UvdjMvaXNzdWVyLW1vY2siLCJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1h                 
           bmNlL3YzL2F1dGgtbW9jayIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l                 
           0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RV                 
           czNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4S                 
           DMydnMzQjVnOFdyN2tjYyJ9.n7KpSfgDZjnajjBhNBFAo5G1Mqxhq2h8JvB7o5_Tj1zB4ZrCM_8VGy1yk8YE1S9                 
           uTyCfUZ9RFtMtM3qwHui9FA'      

[23:27:51] Credential issuer configuration:                                                        ]8;id=615694;file:///tmp/ipykernel_59984/1703696794.py\1703696794.py]8;;\:]8;id=91937;file:///tmp/ipykernel_59984/1703696794.py#21\21]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cre                 
           dential',                                                                                               
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer                 
           -mock/credential_deferred',                                                                             
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework': 

[23:27:52] Authorization server configuration:                                                     ]8;id=911052;file:///tmp/ipykernel_59984/1703696794.py\1703696794.py]8;;\:]8;id=893010;file:///tmp/ipykernel_59984/1703696794.py#24\24]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/au                 
           thorize',                                                                                               
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Perform authorization request and obtain ID token request

In [30]:
from ebsi_wallet.siop_auth.util import (
    perform_authorization, 
    AuthorizationRequestQueryParams,
    get_authorization_response_query_params,
    generate_code_challenge,
    generate_code_verifier
)
import uuid
import json

state = str(uuid.uuid4())
nonce = str(uuid.uuid4())
code_verifier = generate_code_verifier()

authorization_details = [{"type":"openid_credential","format":"jwt_vc","types":credential_offer.credentials[0].get("types"),"locations": ["https://api-conformance.ebsi.eu/conformance/v3/issuer-mock"]}]
redirect_uri = "http://localhost:8080"
client_metadata = {"vp_formats_supported":{"jwt_vp":{"alg":["ES256"]},"jwt_vc":{"alg":["ES256"]}},"response_types_supported":["vp_token","id_token"],"authorization_endpoint":redirect_uri}
authorization_request_query_params = AuthorizationRequestQueryParams(
    response_type="code",
    scope="openid",
    state=state,
    client_id=key_did._did,
    authorization_details=json.dumps(authorization_details, separators=(',', ':')),
    redirect_uri=redirect_uri,
    nonce=nonce,
    code_challenge=generate_code_challenge(code_verifier),
    code_challenge_method="S256",
    client_metadata=json.dumps(client_metadata, separators=(',', ':')),
    issuer_state=credential_offer.grants.get("authorization_code").get("issuer_state")
)

auth_resp = await perform_authorization(auth_server_configuration.authorization_endpoint, authorization_request_query_params)
auth_resp_uri = str(auth_resp).split("Location': '")[1].split("'")[0]

auth_resp_query_params = get_authorization_response_query_params(auth_resp_uri)
console.log("Authorization response query params: ", auth_resp_query_params)

           Authorization response query params:                                                    ]8;id=160986;file:///tmp/ipykernel_59984/3692576654.py\3692576654.py]8;;\:]8;id=863478;file:///tmp/ipykernel_59984/3692576654.py#36\36]8;;\
           AuthorizationResponseQueryParams(                                                                       
               state='65b7db9a-f8be-4ed6-92c5-90d478ac8048',                                                       
               client_id='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                               
               redirect_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                 
           ,                                                                                                       
               response_type='id_token',                                                                           
               response_mode='direct_post',                                                                        
               scope='openid',                                                                                     
               nonce='15eedc04-b03a-4e4d-8a66-83ce53644bb6',                                                       
               request_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/request_uri/b                 
           c6b876e-9a29-47cd-ae02-e937a5440a5d'                                                                    
           )                                                                                                       

## 1.3 Send ID token

In [31]:
from ebsi_wallet.siop_auth.util import send_id_token_response

id_token = key_did.generate_id_token(auth_server_uri=auth_resp_query_params.client_id, nonce=auth_resp_query_params.nonce)
auth_code_response = await send_id_token_response(auth_resp_query_params.redirect_uri, id_token, auth_resp_query_params.state)
auth_code_response = str(auth_code_response).split("Location': '")[1].split("'")[0]
state = parse_query_string_parameters_from_url(auth_code_response).get("state")[0]
auth_code = parse_query_string_parameters_from_url(auth_code_response).get("code")[0]
console.log("Authorization code: ", auth_code)

[23:27:54] Authorization code:  1da99a10-2ee2-4844-84be-77dd449a0701                                ]8;id=713598;file:///tmp/ipykernel_59984/1546471797.py\1546471797.py]8;;\:]8;id=400649;file:///tmp/ipykernel_59984/1546471797.py#8\8]8;;\

## 1.4 Exchange code for access token

In [32]:
from ebsi_wallet.siop_auth.util import exchange_auth_code_for_access_token

token_uri = auth_resp_query_params.client_id + "/token"
access_token_response = await exchange_auth_code_for_access_token(token_uri, key_did._did, auth_code, code_verifier)
console.log("Access token response: ", access_token_response)

[23:27:55] Access token response:                                                                    ]8;id=423941;file:///tmp/ipykernel_59984/474231906.py\474231906.py]8;;\:]8;id=488988;file:///tmp/ipykernel_59984/474231906.py#5\5]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWV               
           ZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjI1YWQ0ZDFmLWI0OTctNDFkZC1hNDhjLTFhNzg2M               
           GViNGZiMyIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudG               
           lhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV               
           1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVy               
           aWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRTYW1lRGVmZXJyZWQiXX1dLCJjX25vbmNlIjoiNjUyMjdhMzgtY               
           jRjNi00ZDcxLWEyMDctYjNlMzlhNjdmMThjIiwiY19ub25jZV9leHBpcmVzX2luIjo4NjQwMCwiY2xpZW50X2lkIj               
           oiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2FqMlZ               
           QMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXRRQUVC               
           RTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIn0sImlzcyI6Imh0d               
           HBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2NrIiwiYXVkIjpbImh0dH               
           BzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvaXNzdWVyLW1vY2siXSwic3ViIjoiZGl               
           kOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2FqMlZQMVM2               
           QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXRRQUVCRTVjU               
           GlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwiaWF0IjoxNjg3MTk3ND               
           c1LCJleHAiOjE2ODcyODM4NzV9.-cUoewRuOlJyPHb0vh9AYDltJd90mmEN4hdDIbR1rIeXpLUMJ4GqsexDr2lM_g               
           pheiw2aP10RoPqEDUGpPFMUg',                                                                              
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWl               
           GbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjU3YmQwYTYzLWE3MTMtNGFiZC1hZDEzLTlkNDQxNjliN               
           TE0YiIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2               
           NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J               
           wdFF3b2FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJB               
           Wmt6eXRRQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwiY               
           XVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2               
           FqMlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXR               
           RQUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwiaWF0Ijox               
           Njg3MTk3NDc1LCJleHAiOjE2ODcxOTc1OTV9.xH57lhUSnVRb_SF-6YKRS4qXgcxA0gOkM7Ope7D0_tOwwYOzBc0z               
           kTzyzbr5tP9DmFYXaNthw0NQQHtG2A4x5A',                                                                    
               c_nonce='65227a38-b4c6-4d71-a207-b3e39a67f18c',                                                     
               c_nonce_expires_in=86400     

## 1.5 Request credential to get acceptance token to initiate deferred flow 

In [33]:
from ebsi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
acceptance_token_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
console.log("Acceptance token response: ", acceptance_token_response)

           Credential request JWT:                                                                  ]8;id=174858;file:///tmp/ipykernel_59984/2195976816.py\2195976816.py]8;;\:]8;id=561955;file:///tmp/ipykernel_59984/2195976816.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtV                
           WjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkRO                
           U1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMz                
           QjVnOFdyN2tjYyN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwdFF3b2Fq                
           MlZQMVM2QWh6bzdSRUZDVDROQlRQWWRRaW5DWmJDY3lvcVdLaTlRMnVFVzM2RE5TWGhDd2lZbkd6NkJBWmt6eXRR                
           QUVCRTVjUGlkQ0duYWRINFNzTERiU1plRzJTRUNocnF2UXBkSzRNazhIMzJ2czNCNWc4V3I3a2NjIiwidHlwIjoi                
           b3BlbmlkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2Nvb                
           mZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg3MjgzODc1LCJpYXQiOjE2ODcxOTc0NzUsImlzcyI6I                
           mRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWU                
           DFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQ                
           kU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsIm5vbmNlIjoiN                
           jUyMjdhMzgtYjRjNi00ZDcxLWEyMDctYjNlMzlhNjdmMThjIn0.eSY-EEPMNuZSTi07IShgrgzhYLvxTO8UzQbyK                
           gdq9mCWT3dJZcKdRLl20niSNI1HS5NSdHeZMHvI1pDJbOu_dg                                                       

[23:27:56] Acceptance token response:                                                               ]8;id=577369;file:///tmp/ipykernel_59984/2195976816.py\2195976816.py]8;;\:]8;id=762692;file:///tmp/ipykernel_59984/2195976816.py#6\6]8;;\
           {'acceptance_token': 'PYChUoHH3E_l40J6RZ3t07w24ahgMC0M9dHXYuJuRIk'}                                     

## 1.6 Request credential using deferred endpoint

In [34]:
from ebsi_wallet.siop_auth.util import send_deferred_credential_request
from time import sleep
import aiohttp
console.log("Waiting for 5 seconds...")
sleep(5)
console.log("Sending deferred credential request...")
credential_response = await send_deferred_credential_request(credential_issuer_configuration.deferred_credential_endpoint, acceptance_token_response.get("acceptance_token"))
print("Credential response: ", credential_response)

           Waiting for 5 seconds...                                                                  ]8;id=799080;file:///tmp/ipykernel_59984/954179285.py\954179285.py]8;;\:]8;id=940043;file:///tmp/ipykernel_59984/954179285.py#4\4]8;;\

[23:28:01] Sending deferred credential request...                                                    ]8;id=166228;file:///tmp/ipykernel_59984/954179285.py\954179285.py]8;;\:]8;id=239747;file:///tmp/ipykernel_59984/954179285.py#6\6]8;;\

Credential response:  {'format': 'jwt_vc', 'credential': 'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNvbmZvcm1hbmNlIzE5YmZlMzg4LTQ2MDgtNDM1OS1iMTM2LWUxMjYzNWE5MjE1NyIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticHRRd29hajJWUDFTNkFoem83UkVGQ1Q0TkJUUFlkUWluQ1piQ2N5b3FXS2k5UTJ1RVczNkROU1hoQ3dpWW5HejZCQVprenl0UUFFQkU1Y1BpZENHbmFkSDRTc0xEYlNaZUcyU0VDaHJxdlFwZEs0TWs4SDMydnMzQjVnOFdyN2tjYyIsImlzcyI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjg3MTk3NDc2LCJleHAiOjE2ODcyODM4NzYsImlhdCI6MTY4NzE5NzQ3NiwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwiaWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlIzE5YmZlMzg4LTQ2MDgtNDM1OS1iMTM2LWUxMjYzNWE5MjE1NyIsInR5cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldFNhbWVEZWZlcnJlZCJdLCJpc3N1ZXIiOiJkaWQ6ZWJzaTp6aEpBUmpQTjY5Y0V0Z1B4SGV